In [105]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [106]:
df = pd.read_csv("cancer_data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# Data Preprocessing

In [107]:
data = df.drop('id', axis=1)

le = LabelEncoder()
data['diagnosis'] = le.fit_transform(data['diagnosis'])

X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

scaler = StandardScaler()
X = scaler.fit_transform(X)

# KNN

In [111]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class KNN:
    def __init__(self, k=3):
        self.k = k
        
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        y_pred = []
        for i in range(len(X_test)):
            distances = np.sqrt(np.sum(np.square(self.X_train - X_test[i, :]), axis=1))
            indices = np.argsort(distances)[:self.k][::-1]
            k_nearest_labels = self.y_train[indices]
            try:
                labels, counts = np.unique(k_nearest_labels, return_counts=True)
                y_pred.append(labels[np.argmax(counts)])
            except:
                y_pred.append(None)
        return y_pred

Xk = data.iloc[:, 2:].values
yk = data.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(Xk, yk, test_size=0.2, random_state=42)

knn = KNN(k=3)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = np.sum(y_pred == y_test) / len(y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9298245614035088


# Naive Bayes

In [109]:
import numpy as np

class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # calculate mean, variance, and prior for each class
        self.mean = np.zeros((n_classes, n_features))
        self.var = np.zeros((n_classes, n_features))
        self.prior = np.zeros(n_classes)
        
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[i, :] = X_c.mean(axis=0)
            self.var[i, :] = X_c.var(axis=0)
            self.prior[i] = X_c.shape[0] / float(n_samples)
            
    def predict(self, X):
        y_pred = []
        for x in X:
            posteriors = []
            for i, c in enumerate(self.classes):
                prior = np.log(self.prior[i])
                likelihood = np.sum(np.log(self.gaussian_pdf(x, self.mean[i], self.var[i])))
                posterior = prior + likelihood
                posteriors.append(posterior)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return np.array(y_pred)
    
    def gaussian_pdf(self, x, mean, var):
        eps = 1e-4
        coeff = 1.0 / np.sqrt(2.0 * np.pi * var + eps)
        exponent = np.exp(-(np.square(x - mean) / (2 * var + eps)))
        return coeff * exponent

In [110]:
nb = NaiveBayes()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print("Accuracy of Naive Bayes: ", accuracy_score(y_test, y_pred))

Accuracy of Naive Bayes:  0.935672514619883
